# Prediction with .h5 file

### Imports

In [1]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import os
import pandas as pd
import numpy as np

import tools

# Root CSV files directory
dirname = "./prediction_data/"

# Frame count
frames = 100

/home/datagroup/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/datagroup/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/datagroup/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/datagroup/anaconda3/lib/python3.7/site-packages/tensorflow/python/f

In [2]:
model = tf.keras.models.load_model('sign_lang_recognition.h5')
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 64)           38656     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 32)           12416     
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense (Dense)                (None, 5)                 165       
Total params: 59,557
Trainable params: 59,557
Non-trainable params: 0
_________________________________________________________________


#### Load the file to predict

In [3]:
listfile = os.listdir(dirname)
data = []
for wordname in listfile:
    if wordname == ".DS_Store":
        continue
    for csv in os.listdir(dirname + wordname):
        filepath = os.path.join(dirname, wordname, csv)
        content = pd.read_csv(filepath, sep=';')
        content = content.reindex(list(range(0, frames)), fill_value=0.0)
        content.fillna(0.0, inplace = True) 
        data.append((wordname, content))
features = [n[1] for n in data]
features = [f.to_numpy() for f in features]
labels = [n[0] for n in data]
x_train=np.array(features)

tokenizer = tools.tokenize(dirname)
print(tokenizer.word_index)
encoded_train=tokenizer.texts_to_sequences([labels])[0]
y_train = to_categorical(encoded_train)
print(y_train)

{'computer': 1, 'hallo': 2, 'welt': 3, 'deutschland': 4}
[[0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]


In [4]:
print(x_train)

[[[0.505633 0.400101 0.451716 ... 0.       0.       0.      ]
  [0.502219 0.403942 0.451105 ... 0.       0.       0.      ]
  [0.505479 0.400694 0.448652 ... 0.       0.       0.      ]
  ...
  [0.       0.       0.       ... 0.       0.       0.      ]
  [0.       0.       0.       ... 0.       0.       0.      ]
  [0.       0.       0.       ... 0.       0.       0.      ]]

 [[0.511228 0.28019  0.125848 ... 0.       0.       0.      ]
  [0.512116 0.282386 0.143416 ... 0.       0.       0.      ]
  [0.51278  0.283563 0.155686 ... 0.       0.       0.      ]
  ...
  [0.       0.       0.       ... 0.       0.       0.      ]
  [0.       0.       0.       ... 0.       0.       0.      ]
  [0.       0.       0.       ... 0.       0.       0.      ]]

 [[0.507547 0.382073 0.213684 ... 0.       0.       0.      ]
  [0.50686  0.383572 0.274671 ... 0.834884 0.737334 0.852904]
  [0.505028 0.391949 0.284391 ... 0.751541 0.714473 0.775634]
  ...
  [0.       0.       0.       ... 0.       0.   

In [5]:
y_pred = model.predict(x_train)

In [6]:
predictions = np.array([np.argmax(pred) for pred in y_pred])
print(predictions)

[4 2 3]
